# ***InfluxDB Time-Series Data Processing with Python***
#### `Name & Surname: Abdulkadir Arslan`

### **1.** I downloaded InfluxDB from [its official website](https://www.influxdata.com/downloads/) onto my native operating system (Windows) and installed it by the instructions given on the website. I started InfluxDB with Windows Powershell and the configuration has been done on the InfluxDB UI. Organization, user, bucket, and API token were created via the web browser.

### **2.** *Installation of the Libraries*

In [1]:
#pip install influxdb_client

In [2]:
#pip install influxdb_client.client.write_api

### **3.** Developing a Python Application for `Moisture` Measurements

#### **3.1** Imports

In [3]:
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
import random
import datetime

#### **3.2** InfluxDB Credentials

In [4]:
client = InfluxDBClient(url="http://localhost:8086", 
                        token="J5pM7sJgjogp9-QByCuA97epf4zxCspI6t1b7pxIqOwE186YvLwz7ljsdXbLCZuX2RAHHn1E8m37f1jrD1jtJw==", 
                        bucket="Big_Data_Task1", 
                        org="FH_Kiel")

write_api = client.write_api(write_options=SYNCHRONOUS)

#### **3.3** Creating Random Measurements

In [5]:
measurement = "moisture_level"
locations = ["living-room", "bathroom", "balcony", "bedroom", "kitchen"]
sensor_types = ["capacitive_sensor", "resistive_sensor", "conductive_sensor"]

def generate_random_moisture_level():
    return random.uniform(0, 100)  

generated_count = 0

for i in range(100):
    timestamp = datetime.datetime.utcnow() - datetime.timedelta(minutes=i)
    moisture_level = generate_random_moisture_level()
    location = random.choice(locations)
    sensor_type = random.choice(sensor_types)

    point = Point("moisture_level")\
        .tag("location", location)\
        .tag("sensor_type", sensor_type)\
        .field("value", moisture_level)\
        .time(timestamp, WritePrecision.NS)

    write_api.write(bucket="Big_Data_Task1", org="FH_Kiel", record=point)
    
    generated_count += 1

confirmation_message = "Creation of {} elements completed!".format(generated_count)
print(confirmation_message)


Creation of 100 elements completed!


#### **3.4** Retrieving the Created Measurements (Extracting and Displaying)

In [6]:
query = '''
    from(bucket: "Big_Data_Task1")
        |> range(start: -10h)
        |> filter(fn: (r) => r["_measurement"] == "moisture_level")
'''

query_api = client.query_api()
result = query_api.query(org="FH_Kiel", query=query)

for table in result:
    for record in table.records:
        time = record.get_time()
        location = record['location']
        sensor_type = record['sensor_type']
        moisture_level = record['_value']
        print(f"Time: {time}, Location: {location}, Sensor Type: {sensor_type}, Moisture Level: {moisture_level}%")


Time: 2024-04-29 17:23:42.619584+00:00, Location: balcony, Sensor Type: capacitive_sensor, Moisture Level: 14.29294174691792%
Time: 2024-04-29 17:40:42.531217+00:00, Location: balcony, Sensor Type: capacitive_sensor, Moisture Level: 48.90652534176007%
Time: 2024-04-29 17:52:42.469642+00:00, Location: balcony, Sensor Type: capacitive_sensor, Moisture Level: 66.96365685221787%
Time: 2024-04-29 18:21:42.300134+00:00, Location: balcony, Sensor Type: capacitive_sensor, Moisture Level: 5.044966979011189%
Time: 2024-04-29 18:32:42.154193+00:00, Location: balcony, Sensor Type: capacitive_sensor, Moisture Level: 26.431929277635092%
Time: 2024-04-29 18:52:51.499410+00:00, Location: balcony, Sensor Type: capacitive_sensor, Moisture Level: 3.561489581728372%
Time: 2024-04-29 18:55:41.762806+00:00, Location: balcony, Sensor Type: capacitive_sensor, Moisture Level: 78.37727179770773%
Time: 2024-04-29 19:10:51.249469+00:00, Location: balcony, Sensor Type: capacitive_sensor, Moisture Level: 17.1191971